In [1]:
import syft as sy
import time

In [2]:
from syft.grid.duet.exchange_ids import DuetCredentialExchanger
from typing import Any as TypeAny
import json

In [3]:
%autoawait
import time
import asyncio
import nest_asyncio
from aries_basic_controller.aries_controller import AriesAgentController

    
# Anyone know how to get env variables into juypter notebooks through docker easily?
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://0.0.0.0:8021"

nest_asyncio.apply()

# Note AttachmentController just extends the basic controller to include the new attachment endpoint
scientist_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [4]:
# Anyone know how to get env variables into juypter notebooks through docker easily?
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://0.0.0.0:8051"

# Note AttachmentController just extends the basic controller to include the new attachment endpoint
dataowner_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [7]:
class AriesCredentialExchanger(DuetCredentialExchanger):
    def __init__(self, agent_controller: TypeAny) -> None:
        super().__init__()
        self.agent_controller = agent_controller
        self.responder_id = None
        self.register_agent_listeners()

    def run(
        self,
        credential: str,
    ) -> str:
        self.credential=credential
        if self.join:
            self._client_exchange(duet_token = credential)
        else: 
            self._server_exchange(duet_token = credential)
            
        
#         self.requester_id = credential
#         self.agent_controller.joiner(self.join)
#         self.agent_controller.send(self.requester_id)
#         while True:
#             if self.agent.has_response():
#                 self.responder_id = self.agent.get_responder_id()

        return self.responder_id
    
    def _client_exchange(self, duet_token):
        
        while True:
            invite = input("♫♫♫ > Duet Partner's Aries Invitation: ")  # nosec
            loop = asyncio.get_event_loop()
            is_ready = False
            try: 
                response = loop.run_until_complete(self.agent_controller.connections.accept_connection(invite))
                print(response["connection_id"])
                print(response)
                connection_id = response["connection_id"]
                is_ready = loop.run_until_complete(self.check_active(connection_id))
                print(is_ready)
            except:
                print("    > Error: Invalid invitatio. Please try again.")
            if is_ready:
                break;
        
        loop.run_until_complete(self.agent_controller.messaging.send_message(connection_id, duet_token))
        
        while True:
            time.sleep(2)
            if self.responder_id:
                break
                
            
                
        
    def _server_exchange(self, duet_token):
        loop = asyncio.get_event_loop()
        response = loop.run_until_complete(self.agent_controller.connections.create_invitation())
        connection_id = response["connection_id"]
        invite_message = json.dumps(response['invitation'])

        print()
        print(
            "♫♫♫ > "

            + "STEP 1:"
            + " Send the aries invitation to your Duet Partner!"
        )
        print()
        print(invite_message)
        while True:
            is_ready = loop.run_until_complete(self.check_active(connection_id))
            if is_ready:
                break
            else: 
                time.sleep(2)

        loop.run_until_complete(self.agent_controller.messaging.send_message(connection_id, duet_token))
        while True:
            time.sleep(2)
            if self.responder_id:
                break

    async def check_active(self, connection_id):
        loop = asyncio.get_event_loop()
        response = loop.run_until_complete(self.agent_controller.connections.get_connection(connection_id))
        return "active" == response["state"]
    
    def register_agent_listeners(self):
        print("REGISTER LISTENERS")
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.agent_controller.listen_webhooks())
        
        listeners = [{
            "handler": self.messages_handler,
            "topic": "basicmessages"
        },{
            "topic": "issue_credential",
            "handler": self.cred_handler
        },{
            "handler": self.connection_handler,
            "topic": "connections"
        },{
            "topic": "present_proof",
            "handler": self.proof_handler
        }]
        self.agent_controller.register_listeners(listeners, defaults=True)
        return
        
    def cred_handler(self, payload):
        print("Handle Credentials")
        exchange_id = payload['credential_exchange_id']
        state = payload['state']
        role = payload['role']
        attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
        print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
        print(f"Offering: {attributes}")
    
    def connection_handler(self, payload):
        print("Connection Handler Called")
        connection_id = payload["connection_id"]
        state = payload["state"]
        print(f"Connection {connection_id} in State {state}")
        
        
    def proof_handler(self, payload):
        print("Handle present proof")
        role = payload["role"]
        pres_ex_id = payload["presentation_exchange_id"]
        state = payload["state"]
        print(f"Role {role}, Exchange {pres_ex_id} in state {state}")
        
    # Receive basic messages
    def messages_handler(self, payload):

        self.responder_id = payload["content"]
        print("Handle Duet ID", duet_id)    
        
    def accept_connection(self, invite):
        # Receive Invitation
        loop = asyncio.get_event_loop()
        response = loop.run_until_complete(self.agent_controller.connections.accept_connection(invitation))
        # Print out accepted Invite and Alice's connection ID
        print("Connection", response)
        return response["connection_id"]
        
    def create_invitation(self, is_duet: bool = False):
        # Create Invitation
        loop = asyncio.get_event_loop()
        invite = loop.run_until_complete(self.agent_controller.connections.create_invitation())
        connection_id = invite["connection_id"]
        invite_message = json.dumps(invite['invitation'])
#         print("Connection ID", dataowner_id)
#         print("Copy Invitation to Data Owner\n")
        return invite_message


In [8]:
dataowner_exchanger = AriesCredentialExchanger(agent_controller=dataowner_controller)
datascientist_exchanger = AriesCredentialExchanger(agent_controller=scientist_controller)

REGISTER LISTENERS
REGISTER LISTENERS


In [9]:
datascientist_exchanger.run('1234')


♫♫♫ > STEP 1: Send the aries invitation to your Duet Partner!

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "8e288bd2-35a4-4611-bcbd-9dd3968e91e9", "serviceEndpoint": "http://7570b5c6ef5a.ngrok.io", "label": "DATASCIENTIST", "recipientKeys": ["14cZQZ4eGTM6oEBDhrR7gJQRgEvU7eKqQH5Ze5npEwRL"]}


KeyboardInterrupt: 

In [9]:
dataowner_exchanger.set_role(True)
dataowner_exchanger.run("1234")

♫♫♫ > Duet Partner's Aries Invitation: {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "f34ecb9b-4009-4e23-af1b-5e82fe21d3f7", "serviceEndpoint": "http://7570b5c6ef5a.ngrok.io", "label": "DATASCIENTIST", "recipientKeys": ["58iEoYhCWiu3mtwsCCwuBNPTyhBdcnomh3HzXWE7A1Xs"]}
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State invitation
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State invitation
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State request
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State request
c4d14984-8307-423e-b95f-5b1517af20ff
{'invitation_mode': 'once', 'initiator': 'external', 'accept': 'manual', 'request_id': '94652ecd-b7bd-45d0-a912-91d71df9b2ff', 'created_at': '2020-12-21 13:51:48.088188Z', 'connection_id': 'c4d14984-8307-423e-b95f-5b1517af20ff', 'updated_at': '2020-12-21 13:51:48.101782Z', 'my_did': 'Hu

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/site-packages/aiohttp/web_protocol.py", line 418, in start
    resp = await task
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/asyncio/futures.py", line 260, in __await__
    yield self  # This tells Task to wait for completion.
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/asyncio/tasks.py", line 349, in __wakeup
    future.result()
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/asyncio/futures.py", line 178, in result
    raise self._exception
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/asyncio/tasks.py", line 280, in __step
    result = coro.send(None)
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/site-packages/aiohttp/web_app.py", line 458, in _handle
    resp = await handler(request)
  File "/home/will/anaconda3/envs/my_syft_env/lib/python3.8/site-packages/aries_basic_contro

Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State response
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State response
Connection Handler Called
Connection 313644ec-6849-46de-a8ec-1cb2fd7f5652 in State active
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State active
Connection Handler Called
Connection c4d14984-8307-423e-b95f-5b1517af20ff in State active
True


KeyboardInterrupt: 

In [19]:
dataowner_exchanger.run("123")

RuntimeError: Session is closed

In [ ]:
await dataowner_exchanger.agent_controller.terminate()
await datascientist_exchanger.agent_controller.terminate()